In [ ]:
import pandas as pd
file_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\3rd Sem\NLP\Assignment_2\data.csv"  # Replace 'path_to_your_file.csv' with the actual path to your CSV file

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.head()


In [214]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Data Preprocessing
# Read the dataset
file_path=r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\3rd Sem\NLP\Assignment_2\data.csv"
data = pd.read_csv(file_path)


In [215]:
data[0:1]

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive


In [216]:
data.loc[data["Sentiment"] == "negative", "Sentiment"] = 2
data.loc[data["Sentiment"] == "neutral", "Sentiment"] = 0
data.loc[data["Sentiment"] == "positive", "Sentiment"] = 1

In [217]:

# Preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    # text = text.lower()
    # # Remove punctuation
    # text = re.sub(r'[^\w\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text.lower())
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    return tokens

# Apply preprocessing to the text data
data['clean_text'] = data['Sentence'].apply(preprocess_text)


In [218]:

# Step 2: Word Embedding Models
# Train CBOW model
cbow_model = Word2Vec(sentences=data['clean_text'], vector_size=100, window=5, sg=0, min_count=1, workers=4)

# Train Skip-gram model
skipgram_model = Word2Vec(sentences=data['clean_text'], vector_size=100, window=5, sg=1, min_count=1, workers=4)


In [219]:
data

,Sentence,Sentiment,clean_text
0,The GeoSolutions technology will leverage Bene...,1,"[geosolutions, technology, leverage, benefon, ..."
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",2,"[esi, lows, 1.50, 2.50, bk, real, possibility]"
2,"For the last quarter of 2010 , Componenta 's n...",1,"[last, quarter, 2010, componenta, 's, net, sal..."
3,According to the Finnish-Russian Chamber of Co...,0,"[according, finnish-russian, chamber, commerce..."
4,The Swedish buyout firm has sold its remaining...,0,"[swedish, buyout, firm, sold, remaining, 22.4,..."
...,...,...,...
5837,RISING costs have forced packaging producer Hu...,2,"[rising, costs, forced, packaging, producer, h..."
5838,Nordic Walking was first used as a summer trai...,0,"[nordic, walking, first, used, summer, trainin..."
5839,"According shipping company Viking Line , the E...",0,"[according, shipping, company, viking, line, e..."
5840,"In the building and home improvement trade , s...",0,"[building, home, improvement, trade, sales, de..."


In [220]:

# Step 3: Neural Network Model for Sentiment Analysis
# Convert text data to numerical vectors using trained word embeddings
def word_to_vector(text, model):
    vector = []
    for word in text:
        if word in model.wv.key_to_index:
            vector.append(model.wv[word])
    return np.mean(vector, axis=0)

data['cbow_vectors'] = data['clean_text'].apply(lambda x: word_to_vector(x, cbow_model))
data['skipgram_vectors'] = data['clean_text'].apply(lambda x: word_to_vector(x, skipgram_model))

# Split data into train and test sets
X_cbow = np.array(data['cbow_vectors'].to_list())
X_skipgram = np.array(data['skipgram_vectors'].to_list())
y = data['Sentiment']

X_train_cbow, X_test_cbow, y_train, y_test = train_test_split(X_cbow, y, test_size=0.2, random_state=42)
X_train_skipgram, X_test_skipgram, _, _ = train_test_split(X_skipgram, y, test_size=0.2, random_state=42)


In [221]:
data

,Sentence,Sentiment,clean_text,cbow_vectors,skipgram_vectors
0,The GeoSolutions technology will leverage Bene...,1,"[geosolutions, technology, leverage, benefon, ...","[-0.087841325, 0.20442438, 0.057850752, -0.014...","[0.07993031, 0.23869692, 0.048818674, -0.06374..."
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",2,"[esi, lows, 1.50, 2.50, bk, real, possibility]","[-0.017208524, 0.041779734, 0.0081582265, -0.0...","[0.019965336, 0.117266014, 0.021185737, -0.018..."
2,"For the last quarter of 2010 , Componenta 's n...",1,"[last, quarter, 2010, componenta, 's, net, sal...","[-0.24729903, 0.25635463, 0.09184723, 0.083167...","[-0.20074609, 0.27741462, 0.19301237, 0.117381..."
3,According to the Finnish-Russian Chamber of Co...,0,"[according, finnish-russian, chamber, commerce...","[-0.14599639, 0.2901071, 0.08635387, 0.0008373...","[0.060563523, 0.2579861, 0.10622661, -0.043819..."
4,The Swedish buyout firm has sold its remaining...,0,"[swedish, buyout, firm, sold, remaining, 22.4,...","[-0.111167245, 0.2420246, 0.06554531, -0.00958...","[-0.04758593, 0.28580222, 0.06357659, 0.021623..."
...,...,...,...,...,...
5837,RISING costs have forced packaging producer Hu...,2,"[rising, costs, forced, packaging, producer, h...","[-0.055991244, 0.13617453, 0.0377705, -0.01089...","[0.029034154, 0.27089432, 0.05073889, -0.04383..."
5838,Nordic Walking was first used as a summer trai...,0,"[nordic, walking, first, used, summer, trainin...","[-0.09274501, 0.123407364, 0.037848614, 0.0206...","[-0.02381691, 0.26700032, 0.047143266, -0.0118..."
5839,"According shipping company Viking Line , the E...",0,"[according, shipping, company, viking, line, e...","[-0.10741274, 0.24980351, 0.0699953, -0.018086...","[0.01966843, 0.27589953, 0.085019514, -0.02793..."
5840,"In the building and home improvement trade , s...",0,"[building, home, improvement, trade, sales, de...","[-0.29684648, 0.23889926, 0.11040356, 0.134824...","[-0.16992375, 0.32099146, 0.2104518, 0.0633464..."


In [222]:

# Define neural network model architecture
def create_model(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [223]:
[print(X_train_cbow.shape, X_train_cbow.dtype) for i in model.inputs]
[print(y_train.shape, y_train.dtype) for i in model.inputs]
[print(X_test_cbow.shape, X_test_cbow.dtype) for o in model.outputs]
[print(y_test.shape, y_test.dtype) for o in model.outputs]
#[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(4673, 100) float32
(4673,) object
(1169, 100) float32
(1169,) object


[None]

In [224]:
y_train = np.array([np.array(val) for val in y_train])
y_test = np.array([np.array(val) for val in y_test])

y_train = tf.cast(y_train , dtype=tf.float32)
y_test = tf.cast(y_test , dtype=tf.float32)

In [225]:

# Train neural network model using CBOW embeddings
model_cbow = create_model(input_shape=(100,))
model_cbow.fit(X_train_cbow, y_train, epochs=100, batch_size=32, validation_split=0.1)


Epoch 1/100


C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3318 - loss: 0.6719 - val_accuracy: 0.4175 - val_loss: 0.6394
Epoch 2/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3498 - loss: 0.6497 - val_accuracy: 0.3829 - val_loss: 0.6026
Epoch 3/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3627 - loss: 0.6421 - val_accuracy: 0.4196 - val_loss: 0.6237
Epoch 4/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3752 - loss: 0.6446 - val_accuracy: 0.3475 - val_loss: 0.5873
Epoch 5/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3823 - loss: 0.6406 - val_accuracy: 0.3946 - val_loss: 0.5835
Epoch 6/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3993 - loss: 0.6385 - val_accuracy: 0.3767 - val_loss: 0.5756
Epoch 7/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3773 - loss: 0.6347 - val_accuracy: 0.3946 - val_loss: 0.5872
Epoch 8/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3840 - loss: 0.6450 - val_accuracy: 0.3767

In [226]:

# Train neural network model using Skip-gram embeddings
model_skipgram = create_model(input_shape=(100,))
model_skipgram.fit(X_train_skipgram, y_train, epochs=100, batch_size=32, validation_split=0.1)


Epoch 1/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3486 - loss: 0.6621 - val_accuracy: 0.4575 - val_loss: 0.6286
Epoch 2/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3949 - loss: 0.6344 - val_accuracy: 0.3779 - val_loss: 0.5849
Epoch 3/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4148 - loss: 0.6271 - val_accuracy: 0.5067 - val_loss: 0.6095
Epoch 4/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4480 - loss: 0.6192 - val_accuracy: 0.4742 - val_loss: 0.5700
Epoch 5/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4522 - loss: 0.6023 - val_accuracy: 0.4062 - val_loss: 0.5348
Epoch 6/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4520 - loss: 0.5737 - val_accuracy: 0.4963 - val_loss: 0.5598
Epoch 7/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4335 - loss: 0.5991 - val_accuracy: 0.5025 - val_loss: 0.5291
Epoch 8/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4588 - loss: 0.5970 - val_accu

In [227]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Assuming binary classification with threshold 0.5
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return acc, precision, recall, f1


In [228]:

acc_cbow, precision_cbow, recall_cbow, f1_cbow = evaluate_model(model_cbow, X_test_cbow, y_test)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [229]:
acc_skipgram, precision_skipgram, recall_skipgram, f1_skipgram = evaluate_model(model_skipgram, X_test_skipgram, y_test)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


C:\Users\ranjan.patra\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:

# Print evaluation results
print("CBOW Model Performance:")
print("Accuracy:", acc_cbow)
print("Precision:", precision_cbow)
print("Recall:", recall_cbow)
print("F1 Score:", f1_cbow)

print("\nSkip-gram Model Performance:")
print("Accuracy:", acc_skipgram)
print("Precision:", precision_skipgram)
print("Recall:", recall_skipgram)
print("F1 Score:", f1_skipgram)


CBOW Model Performance:
Accuracy: 0.3900769888793841
Precision: 0.4621502293983419
Recall: 0.3900769888793841
F1 Score: 0.3296676109036136

Skip-gram Model Performance:
Accuracy: 0.5431993156544055
Precision: 0.5100198030889342
Recall: 0.5431993156544055
F1 Score: 0.5122885667166349
